In [3]:
import torch
from torch import nn
import torch.nn.functional as f
import math

In [6]:
random_torch = torch.rand(4,4)
print(random_torch)

tensor([[0.7675, 0.8423, 0.1810, 0.0604],
        [0.1840, 0.2889, 0.1090, 0.3766],
        [0.8406, 0.6489, 0.7560, 0.6884],
        [0.3519, 0.5613, 0.8185, 0.1589]])


In [9]:
from torch import Tensor
# 将输入词汇表索引转换为指定维度的Embedding

class TokenEmbeeding(nn.Embedding):
    # vocab_size:词汇表大小，d_model:维度
    def __init__(self, vocab_size, d_model):
        super(TokenEmbeeding, self).__init__(vocab_size, d_model, padding_idx=1)

In [ ]:
# 通过位置编码计算输入序列的每个词的正余弦的位置编码
class PositionalEmbedding(nn.Module):
    def __init__(self, d_model, max_len, device):
        super(PositionalEmbedding, self).__init__()
        # positionEmbedding 
        self.encoding = torch.zeros(max_len, d_model, device=device)
        self.encoding.requires_grad = False
        # 序列
        pos = torch.arange(0, max_len, device=device)
        pos = pos.float().unsqueeze(dim=1)# 转化为二维向量
        _2i = torch.arange(0, d_model, step=2, device=device).float()

        self.encoding[0:,0::2] = torch.sin(pos/(10000**(_2i/d_model)))
        self.encoding[0:,1::2] = torch.cos(pos/(10000**(_2i/d_model)))

    # 前向传播
    def forward(self, x):
        batch_size, seq_len = x.size()
        return self.encoding[:seq_len,:] # 返回前seq_len的长度

In [12]:
class TransformerEmbedding(nn.Module):
    def __init__(self, vocab_size, d_model, max_len, drop_pro, device):
        super(TransformerEmbedding, self).__init__()
        self.tok_emb = TokenEmbeeding(vocab_size, d_model)
        self.pos_emb = PositionalEmbedding(d_model, max_len, device)
        self.drop_out = nn.Dropout(p = drop_pro) #随机丢弃一些神经元来缓解过拟合
        
    def forward(self, x):
        tok_emb = self.tok_emb(x)
        pos_emb = self.pos_emb(x)
        return self.drop_out(tok_emb + pos_emb)